<a href="https://colab.research.google.com/github/Smita569/E-commerce_Marketing_and_sales/blob/main/E_commerce_Marketing_and_sales_Business_Case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement 📈

The e-commerce company aims to leverage **data-driven insights** 🧠 to enhance:

*   Customer Acquisition 🤝
*   Customer Retention ✅
*   Revenue Optimization 💰

The following analysis will be conducted to:

*   Understand key business trends 📊
*   Improve decision-making 🤔

In [3]:
# Import libraries

import pandas as pd
import numpy as np

In [4]:
# Upload csv files into dataframe

df_customerData = pd.read_csv('E-commerce_Marketing_and_sales/CustomersData.csv')
df_discountCoupon = pd.read_csv('E-commerce_Marketing_and_sales/Discount_Coupon.csv')
df_marketingSpend = pd.read_csv('E-commerce_Marketing_and_sales/Marketing_Spend.csv')
df_onlineSales = pd.read_csv('E-commerce_Marketing_and_sales/Online_Sales.csv')
df_taxAmount=pd.read_csv('E-commerce_Marketing_and_sales/Tax_amount.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'E-commerce_Marketing_and_sales/CustomersData.csv'

In [28]:
# Read customer data
df_customerData.head()

,CustomerID,Gender,Location,Tenure_Months
0,17850,M,Chicago,12
1,13047,M,California,43
2,12583,M,Chicago,33
3,13748,F,California,30
4,15100,M,California,49


In [ ]:
# Read discount coupon data
df_discountCoupon.head()

,Month,Product_Category,Coupon_Code,Discount_pct
0,Jan,Apparel,SALE10,10
1,Feb,Apparel,SALE20,20
2,Mar,Apparel,SALE30,30
3,Jan,Nest-USA,ELEC10,10
4,Feb,Nest-USA,ELEC20,20


In [26]:
# Read marketing spend data
df_marketingSpend.head()

,Date,Offline_Spend,Online_Spend
0,1/1/2019,4500,2424.50
1,1/2/2019,4500,3480.36
2,1/3/2019,4500,1576.38
3,1/4/2019,4500,2928.55
4,1/5/2019,4500,4055.30


In [1]:
# Read online sales data
df_onlineSales.head()

NameError: name 'df_onlineSales' is not defined

In [ ]:
# Read tax amount data
df_taxAmount.head()

,Product_Category,GST
0,Nest-USA,10%
1,Office,10%
2,Apparel,18%
3,Bags,18%
4,Drinkware,18%


## 💡 Business questions:


1.  **Identify the months with the highest and lowest acquisition rates.** 📈📉 What strategies could be implemented to address the fluctuations and ensure consistent growth throughout the year? 🤔✅